## Sample Test Code & Weights - Setting 1 (CIFAR10)

In [1]:
# Import Modules
import argparse
import os
import math
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Subset
import torchvision
import torchvision.transforms as transforms

import easydict
import numpy as np
from torch.nn import functional as F

from wideresnet import Wide_ResNet, Wide_ResNet_tiny

In [2]:
# GPU Setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Configuration
args = \
easydict.EasyDict({"dataset": 'cifar10',
                   "train_cls": None,
                   "resumepath": './weights/setting1/',
                   "filename": 'sample_weight_ours.pt',
                   "batch_size": 100,
                   "ood_score": "distance",
                   "experiment": 0
})

args.train_cls = np.load('{}known.npy'.format(args.resumepath)).tolist()

In [5]:
# Select Dataset
if args.dataset.lower() == 'cifar10':
    print('Loading {} Dataset...'.format(args.dataset.upper()))
    mean = [x / 255 for x in [125.3, 123.0, 113.9]]
    std = [x / 255 for x in [63.0, 62.1, 66.7]]
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
    image_shape = (3, 32, 32)
    n_class = 10
    test_set = torchvision.datasets.CIFAR10(root='/home/openset/data', train=False, download=True, transform=transform_test)

elif args.dataset.lower() == 'cifar100':
    print('Loading {} Dataset...'.format(args.dataset.upper()))
    mean = [x / 255 for x in [125.3, 123.0, 113.9]]
    std = [x / 255 for x in [63.0, 62.1, 66.7]]
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
    image_shape = (3, 32, 32)
    n_class = 100
    test_set = torchvision.datasets.CIFAR100(root='/home/openset/data', train=False, download=True, transform=transform_test)
    
elif args.dataset.lower() == 'svhn':
    print('Loading {} Dataset...'.format(args.dataset.upper()))
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    image_shape = (3, 32, 32)
    n_class = 10
    test_set = torchvision.datasets.SVHN(root='/home/openset/data', split='test', download=True, transform=transform_test)
    
elif args.dataset.lower() == 'tiny':
    print('Loading {} Dataset...'.format(args.dataset.upper()))
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
    n_class = 200
    image_shape = (3, 64, 64)
    test_set = torchvision.datasets.ImageFolder('/home/openset/data/tiny-imagenet-200/val', transform=transform_test)
        
else:
    raise ValueError("Unsupported Dataset "+args.dataset)

Loading CIFAR10 Dataset...
Files already downloaded and verified


In [6]:
def get_subsets(dataset, train_cls, return_ood=False):
    known, unknown = [], []
    for i in range(len(dataset)):
        current_class = dataset[i][1]
        if current_class in train_cls:
            known.append(i)
        else:
            unknown.append(i)

    ind = Subset(dataset, known)
    ood = Subset(dataset, unknown)
    
    if return_ood == True:
        return ind, ood
    else:
        return ind

In [7]:
# Get Subset of Classes
print('Building In-Class Dataset...: {}'.format(args.train_cls))
if args.experiment == 0:
    ood_cls = list(set(range(n_class)) - set(args.train_cls))
    ind_test_set, ood_test_set = get_subsets(test_set, args.train_cls, return_ood=True)
    
else:
    ind_test_set = test_set
    transform_test = transforms.Compose([#transforms.Resize((32, 32)),
                                         #transforms.CenterCrop(32),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean, std)])
    
    ood_test_set = torchvision.datasets.ImageFolder('/home/openset/ood/iSUN/', transform=transform_test)
num_classes = len(args.train_cls)

Building In-Class Dataset...: [0, 3, 4, 6, 7, 9]


In [8]:
def get_means(num_means, model, shape):
    with torch.no_grad():
        inp = torch.ones(shape).unsqueeze(0)
        penul, _ = model(inp)
        dim = penul.shape[-1]
    
        means = torch.zeros((num_means, dim))
        for i in range(num_means):
            means[i] = torch.randn(dim)
        print("Feature Dimension: {}".format(dim))
    return means, dim

In [9]:
# Create Model
print('Building Model...')
model = Wide_ResNet(40, 2, 0.3, num_classes)
print("Finished: Model Contains {} Parameters".format(sum([p.numel() for p in model.parameters()])))

# Initialize Means (Anchors)
print('Initializing Means...')
means, dim = get_means(num_classes, model, image_shape)
means_np = means.cpu().numpy()
    
means = torch.tensor(means_np, device=device)
inv_cov_stds = torch.log(torch.exp(torch.tensor(1., device=device)) - 1.0)
weights = torch.ones((len(means)), device=device)
    
# Set Initial Parameters
print('Saving Initial Parameters...')
model = model.to(device)
model.initialize(means, inv_cov_stds, weights, grad=False)

Building Model...
| Wide-Resnet 40x2
Finished: Model Contains 2245958 Parameters
Initializing Means...
Feature Dimension: 128
Saving Initial Parameters...


In [10]:
# Load Model
if args.resumepath != '':
    file = args.resumepath + args.filename
    assert os.path.isfile(file)
    checkpoint = torch.load(file, map_location=device)
    model.load_state_dict(checkpoint['model'])
    
del checkpoint
torch.cuda.empty_cache()

In [11]:
ind_test_loader = torch.utils.data.DataLoader(ind_test_set,
                                              batch_size=args.batch_size,
                                              shuffle=False,                                              
                                              num_workers=1,
                                              pin_memory=True,
                                              drop_last=False)

ood_test_loader = torch.utils.data.DataLoader(ood_test_set,
                                              batch_size=args.batch_size,
                                              shuffle=False,
                                              num_workers=1,
                                              pin_memory=True,
                                              drop_last=False)

In [12]:
def reduce_labels(label_tensor, train_cls, device):
    new = []
    for i in label_tensor:
        new.append(train_cls.index(i))
    return torch.tensor(new).to(device)

def reduce_labels_sv(label_tensor, train_cls, device):
    new = []
    for i in label_tensor:
        new.append(len(train_cls))
    return torch.tensor(new).to(device)

In [13]:
count = 0
score = []
true = []
y_true = []
y_pred = []

with torch.no_grad():
    for x, y in ind_test_loader:
        model.eval()
        x = x.to(device)
        y = y.to(device)
        y = reduce_labels(y, args.train_cls, device)

        z_all, logits = model(x)
        
        if args.ood_score == 'distance':
            distance = torch.norm(z_all.unsqueeze(1) - model.means, p=2, dim=2)**2
            min_score, _ = torch.min(distance, dim=1)
            logits = -distance/2
            
        elif args.ood_score == 'energy':
            min_score = -torch.logsumexp(logits, dim=1)
        
        else:
            prob = F.softmax(logits, dim=1)
            entropy = (prob * torch.log(prob)).sum(1)
            min_score, _ = -torch.max(prob, dim=1)
            #min_dist = min_dist * torch.norm(z_all, dim=1)
            #min_dist = entropy
            
        preds_all = torch.argmax(logits, dim=1)
        count += (preds_all == y).sum()
        y_true = np.append(y_true, y.detach().cpu().numpy())
        y_pred = np.append(y_pred, preds_all.detach().cpu().numpy())

        score = np.append(score, min_score.detach().cpu().numpy())
        true = np.append(true, np.zeros_like(min_score.detach().cpu().numpy()))

    print("Closed-set Classification Acc: {}".format(int(count.cpu())/len(ind_test_loader.dataset)))

    for x, y in ood_test_loader:
        model.eval()
        x = x.to(device)
        y = y.to(device)
        y = reduce_labels_sv(y, args.train_cls, device)

        z_all, logits = model(x)

        if args.ood_score == 'distance':
            distance = torch.norm(z_all.unsqueeze(1) - model.means, p=2, dim=2)**2
            min_score, _ = torch.min(distance, dim=1)
            logits = -distance/2
            
        elif args.ood_score == 'energy':
            min_score = -torch.logsumexp(logits, dim=1)
        
        else:
            prob = F.softmax(logits, dim=1)
            entropy = (prob * torch.log(prob)).sum(1)
            min_score, _ = -torch.max(prob, dim=1)
            #min_score = min_score * torch.norm(z_all, dim=1)
            #min_score = entropy
            
        preds_all = torch.argmax(logits, dim=1)
        count += (preds_all == y).sum()
        y_true = np.append(y_true, y.detach().cpu().numpy())
        y_pred = np.append(y_pred, preds_all.detach().cpu().numpy())

        score = np.append(score, min_score.detach().cpu().numpy())
        true = np.append(true, np.ones_like(min_score.detach().cpu().numpy()))

Closed-set Classification Acc: 0.9728333333333333


In [14]:
from sklearn.metrics import roc_auc_score, average_precision_score
print("AUROC: {}".format(roc_auc_score(true, score)))

AUROC: 0.9446843541666667


In [15]:
import copy
from sklearn import metrics
threshold = np.percentile(score[len(ind_test_loader.dataset):], 10)
index = np.where(score > threshold)
y_pred2 = copy.deepcopy(y_pred)
y_pred2[index] = 6

print("OSCR at FPR of 10^-1: {}".format((y_true[:len(ind_test_loader.dataset)] == y_pred2[:len(ind_test_loader.dataset)]).sum()/len(ind_test_loader.dataset)))

OSCR at FPR of 10^-1: 0.8685
